In [1]:
%matplotlib inline
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import itertools
sns.set_style('darkgrid')
colors = sns.color_palette()

## Simple ODE in Tensorflow

This is the true $y'$.

In [2]:
def odefunc(y, t):
    true_A = np.array([[-0.1, 2.0], [-2.0, -0.1]])
    return np.matmul(y**3, true_A)

This is an approximation of $y'$ using a simple MLP.

In [3]:
def odefunc_tf(y, t):
    with tf.variable_scope('odefunc', reuse=tf.AUTO_REUSE):
        y = tf.reshape(y, (-1, 2))
        z = tf.layers.dense(y**3, 50, activation=tf.nn.tanh)
        z = tf.layers.dense(z, 2, activation=None)
    return z

In [9]:
def draw_plot(t, true_y, pred_y, step=0):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlabel('t')
    ax.set_ylabel('x,y')
    plt.title('%d' % step)
    color=colors[4]
    ax.plot(t, true_y[:, 0, 0], '-', color=color, label='true x')
    ax.plot(t, pred_y[:, 0, 0], '--', color=color, label='est. x')
    color=colors[5]
    ax.plot(t, true_y[:, 0, 1], '-', color=color, label='true y')
    ax.plot(t, pred_y[:, 0, 1], '--', color=color, label='est. y')
    ax.set_xlim(t.min(), t.max())
    ax.set_ylim(-2, 2)
    ax.legend(loc='upper right', shadow=True)
    plt.tight_layout()
    plt.savefig('imgs/step_{:d}.jpg'.format(step), bbox_inches='tight')
    plt.close()

In [5]:
batch_time = 10
batch_size = 20
y0 = np.array([2., 0.])
t = np.linspace(0., 25., 1000)
true_y = odeint(odefunc, y0, t)

def get_batch():
    s = np.random.choice(np.arange(1000 - batch_time), batch_size, replace=False)
    batch_y0 = true_y[s]
    batch_t = t[:batch_time]
    batch_y = np.array([true_y[s + i] for i in range(batch_time)])
    return batch_y0, batch_t, batch_y

In [6]:
y0_tf = tf.placeholder(tf.float32, shape=(None, 2), name='initial_value')
t_tf = tf.placeholder(tf.float32, shape=(None,), name='steps')
target_y = tf.placeholder(tf.float32, shape=(batch_time, batch_size, 2), name='target_values')
pred_y = tf.contrib.integrate.odeint(odefunc_tf, y0_tf, t_tf)

In [7]:
loss = tf.reduce_mean(tf.abs(target_y - pred_y))
optim_op = tf.train.RMSPropOptimizer(1e-3).minimize(loss)
init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()

In [10]:
sess.run(init_op)
imgidx = 1
for i in range(1, 1000 + 1):
    batch_y0, batch_t, batch_y = get_batch()
    _, loss_np = sess.run([optim_op, loss], feed_dict={y0_tf: batch_y0,
                                                      t_tf: batch_t,
                                                      target_y: batch_y})
    if i % 100 == 0 or imgidx == 1:
        print(i, loss_np)
        batch_pred_y = sess.run(pred_y, feed_dict={y0_tf: np.reshape(y0, (1, 2)), t_tf: t})
        draw_plot(t, true_y[:, None, :], batch_pred_y, step=imgidx)
        imgidx += 1

1 0.07794231
100 0.07108314
200 0.008024567
300 0.006438549
400 0.0058195544
500 0.007894609
600 0.006762387
700 0.012875323
800 0.0074920463
900 0.0067219674
1000 0.0010826781
